In [1]:
# 调用 openclaw-llm-bridge `/v1/chat/completions`

# 使用在 `/admin/keys` 创建的 `api_key`（如 `sk-my-user-key-001`）请求头带 `Authorization: Bearer <api_key>` 即可。

In [2]:
import requests

BASE_URL = "http://127.0.0.1:8000"
API_KEY = "sk-my-user-key-001"  # 与 POST /admin/keys 创建时一致

# 非流式调用
resp = requests.post(
    f"{BASE_URL}/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
    },
    json={
        "model": "gpt-5-nano",
        "messages": [
            {"role": "user", "content": "你好，用一句话介绍你自己。"},
        ],
        "stream": False,
    },
    timeout=60,
)
resp.raise_for_status()
data = resp.json()
print("usage:", data.get("usage"))
print("content:", data["choices"][0]["message"]["content"])

usage: {'prompt_tokens': 14, 'completion_tokens': 421, 'total_tokens': 435}
content: 我是OpenAI的语言模型ChatGPT，能用多种语言回答问题、提供信息，并协助写作与学习。


In [3]:
# 流式调用 (SSE)
resp = requests.post(
    f"{BASE_URL}/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
    },
    json={
        "model": "gpt-5-nano",
        "messages": [
            {"role": "user", "content": "数到 5，每行一个数字。"},
        ],
        "stream": True,
    },
    timeout=60,
    stream=True,
)
resp.raise_for_status()
for line in resp.iter_lines():
    if line and line.startswith(b"data: "):
        chunk = line[6:]
        if chunk.strip() == b"[DONE]":
            break
        import json
        d = json.loads(chunk)
        delta = (d.get("choices") or [{}])[0].get("delta") or {}
        if delta.get("content"):
            print(delta["content"], end="", flush=True)
print()

1
2
3
4
5
